In [ ]:
import os
import numpy as np
import pickle
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
%matplotlib inline

import nept

from loading_data import get_data
from utils_maze import find_zones, speed_threshold

cwd = os.getcwd()
pickle_filepath = os.path.join(cwd, "cache", "pickled")
output_filepath = os.path.join(cwd, "plots", "intermediate")

from analyze_decode import get_decoded

In [ ]:
from run import spike_sorted_infos, info

session = info.r063d2

neurons_filename = session.session_id + '_neurons.pkl'
pickled_neurons = os.path.join(pickle_filepath, neurons_filename)
with open(pickled_neurons, 'rb') as fileobj:
    neurons = pickle.load(fileobj)

track_times = ['phase1', 'phase2', 'phase3', 'tracks']
pedestal_times = ['pauseA', 'pauseB', 'prerecord', 'postrecord']
    
experiment_time = "phase3"
t_start = session.task_times[experiment_time].start+0.0125
t_stop = session.task_times[experiment_time].stop
#t_start, t_stop = (5580, 5630)

args = dict(speed_limit=0.4,
            min_swr=3,
            min_neurons=2,
            min_spikes=1,
            t_start=t_start,
            t_stop=t_stop,
            neurons=neurons,
            shuffle_id=False,
            info=session,
            normalized=False,
            sequence_speed=5.,
            sequence_len=3,
            min_epochs=3)
if experiment_time in track_times:
    args["run_time"] = True

In [ ]:
def analyze(window_size, window_advance, gaussian_std):
    _args = args.copy()
    _args["window"] = window_size
    _args["dt"] = window_advance
    _args["gaussian_std"] = gaussian_std
    d = {}

    d['decoded'], d['decoded_epochs'], d['errors'], d['position'] = get_decoded(**_args)
    
    print('window %.4f advancing in %.4f with %s std: %.3f' % (
        window_size, window_advance, gaussian_std, np.mean(d['errors'])))
    
    return d

In [ ]:
nooverlap = analyze(0.025, 0.025, None)
overlap = analyze(0.025, 0.0125, None)

In [ ]:
print(overlap['errors'][:20])
print(nooverlap['errors'][:20])

In [ ]:
nooverlap_offset = analyze(0.025, 0.025, None)

In [ ]:
print(nooverlap_offset['errors'][:20])

In [ ]:
nooverlap = analyze(0.0125, 0.0125, 0.0075)
overlap = analyze(0.025, 0.0125, None)

In [ ]:
print(nooverlap["errors"].shape)
print(overlap["errors"].shape)

In [ ]:
def hist(d):
    plt.figure()
    plt.hist(d["errors"])

hist(nooverlap)
hist(overlap)

In [ ]:
def p(d, left, right):
    t = d['decoded'].time
    error = d['errors']
    errorsig = nept.AnalogSignal(error, t).time_slice(left, right)
    print("n_points: %d, mean error: %.3f" % (
        errorsig.n_samples, np.mean(errorsig.data)))
    plt.plot(t, error, ls="none", marker=".")
    plt.xlim(left, right)
    
def comp(d1, d2, left, right):
    plt.figure()
    p(d1, left, right)
    p(d2, left, right)

In [ ]:
comp(overlap, nooverlap, 5400, 8200)

In [ ]:
comp(both, nooverlap, 5400, 8200)

In [ ]:
comp(overlap, nooverlap, t_start+20, t_start+120)